In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from matplotlib import pyplot

In [2]:
# pd.set_option('display.max_columns', None)

df = pd.read_csv('../../BPIC_Classification/processed_dataset/P_BPIC17_concat_agg.csv')
df.head()

,"('ApplicationType', 'New credit')","('ApplicationType', 'Limit raise')","('LoanGoal', 'Existing loan takeover')","('LoanGoal', 'Home improvement')","('LoanGoal', 'Car')","('LoanGoal', 'Other, see explanation')","('LoanGoal', 'Remaining debt home')","('LoanGoal', 'Not speficied')","('LoanGoal', 'Unknown')","('LoanGoal', 'Caravan / Camper')",...,"('lifecycle:transition', 'ate_abort')","('lifecycle:transition', 'resume')","('Accepted', 'missing')","('Accepted', 'True')","('Accepted', 'False')","('Selected', 'missing')","('Selected', 'True')","('Selected', 'False')",time:timestamp,label
0,40,0,40,0,0,0,0,0,0,0,...,5,2,9,31,0,9,31,0,2016-01-08 16:29:29.766300160,0
1,31,0,0,31,0,0,0,0,0,0,...,1,4,10,0,21,10,0,21,2016-01-04 07:00:38.533548288,1
2,45,0,0,45,0,0,0,0,0,0,...,4,3,8,37,0,8,29,8,2016-01-08 05:07:21.250910976,0
3,84,0,0,0,84,0,0,0,0,0,...,4,18,19,0,65,19,55,10,2016-01-18 14:13:48.282452480,1
4,21,0,0,21,0,0,0,0,0,0,...,1,1,8,13,0,8,0,13,2016-01-06 23:45:36.994237952,1


In [3]:
df['label'].value_counts()

1    18621
0    12792
Name: label, dtype: int64

In [4]:
class AccuracyScore:

    def __init__(self, X_train, y_train, X_test, y_test):
        self.self = self
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def preprocessing(self):
        self['time:timestamp'] = pd.to_datetime(self['time:timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
        self['time:timestamp'] = (self['time:timestamp'] - self['time:timestamp'].min()) / np.timedelta64(1,'M')
        
        return self
    
    def column_uniquify(self):
        cols = pd.Series(self.columns)
        dup_count = cols.value_counts()
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) for i in range(1, dup_count[dup]+1)]
        self.columns = cols
        return self.columns

    def data_split(self):
        X = df.drop('label', axis=1)
        y = df['label']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023, shuffle=True)
        
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        return X_train, y_train, X_test, y_test        
    
    def imbalance(self, X_train, y_train):
        sm = SMOTE(random_state=1)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
        return X_train_res, y_train_res
    
    def DT(self, X_train, y_train, X_test, y_test):
        clf_dt = tree.DecisionTreeClassifier(max_depth=2, random_state=2023, max_features='log2')
        clf_dt = clf_dt.fit(X_train, y_train)
        predictions = clf_dt.predict(X_test)
        accuracy = clf_dt.score(X_test, y_test)
        print('DT Accuracy:', accuracy)
        print('DT AUC Score {}'.format(roc_auc_score(y_test, predictions)))

    def RF(self, X_train, y_train, X_test, y_test):
        clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=2023)
        clf_rf = clf_rf.fit(X_train, y_train)
        predictions = clf_rf.predict(X_test)
        accuracy = clf_rf.score(X_test, y_test)
        print('RF Accuracy:', accuracy)
        print('RF AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def XGB(self, X_train, y_train, X_test, y_test):
        xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=2023)
        xgb.fit(X_train, y_train)
        xgb_pred = xgb.predict(X_test)

        accuracy = accuracy_score(y_test, xgb_pred)
        precision = precision_score(y_test, xgb_pred)
        recall = recall_score(y_test, xgb_pred)
        f1 = f1_score(y_test, xgb_pred)

        print('XGB Accuracy:', accuracy)
        print('XGB AUC Score {}'.format(roc_auc_score(y_test, xgb_pred)))
    
    def LGBM(self, X_train, y_train, X_test, y_test):
        clf_lgbm = LGBMClassifier(random_state=2023)
        clf_lgbm = clf_lgbm.fit(X_train, y_train)
        predictions = clf_lgbm.predict(X_test)
        accuracy = clf_lgbm.score(X_test, y_test)
        print('LGBM Accuracy:', accuracy)
        print('LGBM AUC Score {}'.format(roc_auc_score(y_test, predictions)))

In [5]:
df = AccuracyScore.preprocessing(df)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.columns = AccuracyScore.column_uniquify(df)
X_train, y_train, X_test, y_test = AccuracyScore.data_split(df)
# X_train, y_train = AccuracyScore.imbalance(df, X_train, y_train)

In [6]:
AccuracyScore.DT(df, X_train, y_train, X_test, y_test)
AccuracyScore.RF(df, X_train, y_train, X_test, y_test)
AccuracyScore.XGB(df, X_train, y_train, X_test, y_test)
AccuracyScore.LGBM(df, X_train, y_train, X_test, y_test)

DT Accuracy: 0.7452249575551783
DT AUC Score 0.7817362131013376
RF Accuracy: 0.9731536502546689
RF AUC Score 0.9700932579944768
XGB Accuracy: 0.985144312393888
XGB AUC Score 0.9826754263035752
LGBM Accuracy: 0.9852504244482173
LGBM AUC Score 0.982681312151754
